In [ ]:
import sys
# setting path
sys.path.append('../')

import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'

In [ ]:
MAIN_PATH = "/work/work_fran/Deep_Var_BCD/"
RESULTS_PATH = MAIN_PATH + "results/metrics/old/"
hyperparam_name = "theta_val"
hyperparam_show_name = "theta"

In [ ]:
def highlight_columns(df, cols_idx = None, mode="min"):
    if cols_idx is None:
        cols = df.columns
    else:
        cols = df.columns[cols_idx]
    new_df = df.copy()
    for i in range(len(cols)):
        new_col = ['{:5f}'.format(a) for a in df[f"{cols[i]}"]]
        # Bold max value in each column
        if mode == "max":
            bold_idx = df[f"{cols[i]}"].argmax()
        elif mode == "min":
            bold_idx = df[f"{cols[i]}"].argmin()
        new_col[bold_idx] = "\\mathbf{" + new_col[bold_idx] + "}"
        new_col = ["$" + s + "$" for s in new_col]
        new_df = new_df.assign(**{cols[i]: new_col})
    return new_df

# COMPARATIVA GLOBAL

In [ ]:
path = RESULTS_PATH + "results_camelyon.csv"
df_cam = pd.read_csv(path)
df_cam = df_cam.drop(columns=["patch_size", "n_samples"]+[c for c in df_cam.columns if "loss" in c])
path = RESULTS_PATH + "results_wssb.csv"
df_wssb = pd.read_csv(path)
df_wssb= df_wssb.drop(columns=["patch_size", "n_samples"]+[c for c in df_wssb.columns if ("loss" in c) or ("rec" in c)])
new_df_wssb = df_wssb[["mnet_name", "pretraining_epochs", "sigmaRui_sq", hyperparam_name]]
new_df_wssb["wssb_mean_test_psnr_gt_h"] = df_wssb[[c for c in df_wssb.columns if "psnr_gt_h" in c]].mean(axis=1)
new_df_wssb["wssb_mean_test_psnr_gt_e"] = df_wssb[[c for c in df_wssb.columns if "psnr_gt_e" in c]].mean(axis=1)
new_df_wssb["wssb_mean_test_psnr_gt"] = (new_df_wssb[["wssb_mean_test_psnr_gt_h", "wssb_mean_test_psnr_gt_e"]]).mean(axis=1)
new_df_wssb["wssb_mean_test_ssim_gt_h"] = df_wssb[[c for c in df_wssb.columns if "ssim_gt_h" in c]].mean(axis=1)
new_df_wssb["wssb_mean_test_ssim_gt_e"] = df_wssb[[c for c in df_wssb.columns if "ssim_gt_e" in c]].mean(axis=1)
new_df_wssb["wssb_mean_test_ssim_gt"] = (new_df_wssb[["wssb_mean_test_ssim_gt_h", "wssb_mean_test_ssim_gt_e"]]).mean(axis=1)

In [ ]:
df = pd.merge(df_cam, new_df_wssb, on=["mnet_name", "pretraining_epochs", "sigmaRui_sq", "theta_val"])
df = df.drop(columns=[c for c in df.columns if ("gt_h" in c) or ("gt_e" in c)])

df[hyperparam_name] = df[hyperparam_name].astype(str)
df = highlight_columns(df, cols_idx=[4], mode="min")
df = highlight_columns(df, cols_idx=[5, 6, 7, 8], mode="max")
#df = highlight_columns(df, cols_idx=[2], mode="min")
#df = highlight_columns(df, cols_idx=[3, 4, 5, 6], mode="max")

df = df.sort_values(by=["mnet_name", "pretraining_epochs", "sigmaRui_sq", "theta_val"])

df.columns = ["Mnet", "Pre. epochs", f"$\\theta$", "$(\sigma_s^Rui)^2$", "Rec. MSE (Camelyon)", "Rec. PSNR (Camelyon)", "Rec. SSIM (Camelyon)", "GT PSNR (WSSB)", "GT SSIM (WSSB)"]

df.set_index(["Mnet", "Pre. epochs", "$(\sigma_s^Rui)^2$", f"$\\theta$"], inplace=True)

#df

In [ ]:
df = pd.merge(df_cam, new_df_wssb, on=["mnet_name", "pretraining_epochs", "sigmaRui_sq", "theta_val"])
df = df.drop(columns=[c for c in df.columns if ("gt_h" in c) or ("gt_e" in c)])
df = df.drop(df[df['sigmaRui_sq'] != 0.05].index)
df = df.drop(df[df['mnet_name'] != 'mobilenetv3s'].index)

df[hyperparam_name] = df[hyperparam_name].astype(str)

In [ ]:
df = highlight_columns(df, cols_idx=[4], mode="min")
df = highlight_columns(df, cols_idx=[5, 6, 7, 8], mode="max")

df = df.sort_values(by=["mnet_name", "pretraining_epochs", "sigmaRui_sq", "theta_val"])
df.columns = ["Mnet", "Pre. epochs", f"$\\theta$", "$(\sigma_s^Rui)^2$", "Rec. MSE (Camelyon)", "Rec. PSNR (Camelyon)", "Rec. SSIM (Camelyon)", "GT PSNR (WSSB)", "GT SSIM (WSSB)"]
df.set_index(["Mnet", "Pre. epochs", "$(\sigma_s^Rui)^2$", f"$\\theta$"], inplace=True)

df

In [ ]:
styler = df.style
#styler = styler.hide(level=0, axis=0)  
s = styler.to_latex(
    column_format=f"{ ''.join(['c' for i in range(len(df.columns)+2)]) }", clines="skip-last;data", 
    hrules = True, position_float="centering", multirow_align="c", sparse_index="True", 
    caption="Summary of results.", label="tab:results_summary",
    )
print(s)

## RECONSTRUCTION MSE

In [ ]:
path = RESULTS_PATH + "results_camelyon.csv"
df = pd.read_csv(path)
df = df.drop(df[df['sigmaRui_sq'] != 0.05].index)
df = df.drop(df[df['mnet_name'] != 'mobilenetv3s'].index)
df = df.drop(df[df['pretraining_epochs'] != 1.0].index)
df = df.drop(columns=["pretraining_epochs", "mnet_name", "sigmaRui_sq", "patch_size", "n_samples", "camelyon_test_loss", "camelyon_test_loss_mse", "camelyon_test_loss_kl"])
df[hyperparam_name] = df[hyperparam_name].astype(str)
df = highlight_columns(df, cols_idx=[1], mode="min")
df = highlight_columns(df, cols_idx=[2, 3], mode="max")
df = df.sort_values(by=["theta_val"])
df.set_index(["theta_val"], inplace=True)
#df.columns = ["Reconstruction MSE", "Reconstruction PSNR", "Reconstruction SSIM"]
#df.index.names = ["$\\theta$"]
df_cam = df
df_cam.transpose()

In [ ]:
path = RESULTS_PATH + "results_wssb.csv"
df = pd.read_csv(path)
df = df.drop(df[df['sigmaRui_sq'] != 0.05].index)
df = df.drop(df[df['mnet_name'] != 'mobilenetv3s'].index)
df = df.drop(df[df['pretraining_epochs'] != 1.0].index)
df = df.drop(columns=["pretraining_epochs", "mnet_name", "sigmaRui_sq", "patch_size", "n_samples"] + [c for c in df.columns if "loss" in c])
#df = df.drop(columns=[c for c in df.columns if "gt" in c])
df[hyperparam_name] = df[hyperparam_name].astype(str)

df["test_mse_rec"] = (df["wssb_Lung_test_mse_rec"] + df["wssb_Breast_test_mse_rec"] + df["wssb_Colon_test_mse_rec"])/3.0
df = df.drop(columns=["wssb_Lung_test_mse_rec", "wssb_Breast_test_mse_rec", "wssb_Colon_test_mse_rec"])
df["test_ssim_rec"] = (df["wssb_Lung_test_ssim_rec"] + df["wssb_Breast_test_ssim_rec"] + df["wssb_Colon_test_ssim_rec"])/3.0
df = df.drop(columns=["wssb_Lung_test_ssim_rec", "wssb_Breast_test_ssim_rec", "wssb_Colon_test_ssim_rec"])
df["test_psnr_rec"] = (df["wssb_Lung_test_psnr_rec"] + df["wssb_Breast_test_psnr_rec"] + df["wssb_Colon_test_psnr_rec"])/3.0
df = df.drop(columns=["wssb_Lung_test_psnr_rec", "wssb_Breast_test_psnr_rec", "wssb_Colon_test_psnr_rec"])

df["test_psnr_gt"] = (df["wssb_Lung_test_psnr_gt"] + df["wssb_Breast_test_psnr_gt"] + df["wssb_Colon_test_psnr_gt"])/3.0
df = df.drop(columns=["wssb_Lung_test_psnr_gt", "wssb_Breast_test_psnr_gt", "wssb_Colon_test_psnr_gt"])
df["test_ssim_gt"] = (df["wssb_Lung_test_ssim_gt"] + df["wssb_Breast_test_ssim_gt"] + df["wssb_Colon_test_ssim_gt"])/3.0
df = df.drop(columns=["wssb_Lung_test_ssim_gt", "wssb_Breast_test_ssim_gt", "wssb_Colon_test_ssim_gt"])
df = df.drop(columns=[c for c in df.columns if ("gt_e" in c) or ("gt_h" in c)])
df = df.drop(columns=["wssb_Lung_test_mse_gt", "wssb_Breast_test_mse_gt", "wssb_Colon_test_mse_gt"])

df = highlight_columns(df, cols_idx=[1], mode="min")
df = highlight_columns(df, cols_idx=[2, 3, 4, 5], mode="max")

df = df.sort_values(by=["theta_val"])
df.set_index(["theta_val"], inplace=True)
#df.columns = ["Reconstruction MSE", "Reconstruction PSNR", "Reconstruction SSIM", "Ground-truth PSNR", "Ground-truth SSIM"]
#df.index.names = ["$\\theta$"]
df_wssb = df
df_wssb.transpose()

In [ ]:
df = pd.concat([df_cam, df_wssb], axis=1)
df = df.drop(columns=[c for c in df.columns if ("psnr" in c) or ("ssim" in c)])
df.columns = ["Camelyon Reconstruction MSE", "WSSB Reconstruction MSE"]
df = df.transpose()
df

In [ ]:
styler = df.style
#styler = styler.hide(level=0, axis=0)  
s = styler.to_latex(
    column_format=f"{ ''.join(['c' for i in range(len(df.columns)+2)]) }", clines="skip-last;data", 
    hrules = True, position_float="centering", multirow_align="c", sparse_index="True", 
    caption="Results on Camelyon test dataset.", label="tab:results_camelyon",
    )
print(s)
# Fix header

## Per-stain per-organ metrics

In [ ]:
HYPERPARAM_VAL = 1.0
PRETRAINING_EPOCHS = 0

In [ ]:
path = RESULTS_PATH + "results_wssb.csv"
df = pd.read_csv(path)
#theta_list = list(df["theta_val"])
#pe_list = list(df["pretraining_epochs"])
df = df.drop(columns=[c for c in df.columns if "loss" in c])
df = df[df["pretraining_epochs"] == PRETRAINING_EPOCHS] 
df = df[df[hyperparam_name] == HYPERPARAM_VAL]
df = df[df["mnet_name"] == "resnet18ft"]
df = df[[c for c in df.columns if "psnr_gt" in c]]
df["Name"] = [f"BCDnet p.e.={PRETRAINING_EPOCHS}, $\\theta={HYPERPARAM_VAL}$"]
df = df[["Name"] + list(df.columns[:-1])]
df = df.append({"Name" : "BCDnet (old)", 
    "wssb_Lung_test_psnr_gt_h" : 27.06, "wssb_Lung_test_psnr_gt_e" : 25.69, 
    "wssb_Breast_test_psnr_gt_h" : 21.81, "wssb_Breast_test_psnr_gt_e" : 24.53, 
    "wssb_Colon_test_psnr_gt_h" : 20.32, "wssb_Colon_test_psnr_gt_e" : 21.70}, ignore_index=True)
df = df.append({"Name" : "BKSVD", 
    "wssb_Lung_test_psnr_gt_h" : 32.67, "wssb_Lung_test_psnr_gt_e" : 30.61, 
    "wssb_Breast_test_psnr_gt_h" : 32.20, "wssb_Breast_test_psnr_gt_e" : 29.43, 
    "wssb_Colon_test_psnr_gt_h" : 34.08, "wssb_Colon_test_psnr_gt_e" : 33.32}, ignore_index=True)

df["wssb_mean_test_psnr_gt_h"] = df[[c for c in df.columns if "psnr_gt_h" in c]].mean(axis=1)
df["wssb_mean_test_psnr_gt_e"] = df[[c for c in df.columns if "psnr_gt_e" in c]].mean(axis=1)
df["wssb_mean_test_psnr_gt"] = df[["wssb_mean_test_psnr_gt_h", "wssb_mean_test_psnr_gt_e"]].mean(axis=1)
df = highlight_columns(df, cols_idx=[1,2,3,4,5,6,7,8,9], mode="max")
df = df.transpose()
df.columns = df.iloc[0]
df = df.drop(df.index[0])
df.reset_index(inplace=True)
df.columns.name = None
df = df.drop(columns=["index"])
df["Organ"] = ["Lung", "Lung", "Breast", "Breast", "Colon", "Colon", "Mean", "Mean", "Mean"]
df["Stain"] = ["H", "E", "H", "E", "H", "E", "H", "E", "Mean"]

df.set_index(["Organ", "Stain"], inplace=True)
df

In [ ]:
styler = df.style
s = styler.to_latex(
    column_format=f"{ ''.join(['c' for i in range(len(df.columns)+2)]) }", clines="skip-last;data", 
    hrules = True, position_float="centering", multirow_align="c", sparse_index="True", 
    caption="Ground truth PSNR on WSSB dataset.", label="tab:gt_psnr_wssb",
    )
print(s)

In [ ]:
path = RESULTS_PATH + "results_wssb.csv"
df = pd.read_csv(path)
df = df.drop(columns=[c for c in df.columns if "loss" in c])
df = df[df["pretraining_epochs"] == PRETRAINING_EPOCHS] 
df = df[df[hyperparam_name] == HYPERPARAM_VAL]
#df = df[df["pretraining_epochs"] == 0] 
df = df[[c for c in df.columns if "ssim_gt" in c]]
df["Name"] = [f"BCDnet p.e.={PRETRAINING_EPOCHS}, $\\theta={HYPERPARAM_VAL}$"]
df = df[["Name"] + list(df.columns[:-1])]
df = df.append({"Name" : "BCDnet (old)", 
    "wssb_Lung_test_ssim_gt_h" : 0.9518, "wssb_Lung_test_ssim_gt_e" : 0.8201, 
    "wssb_Breast_test_ssim_gt_h" : 0.8962, "wssb_Breast_test_ssim_gt_e" : 0.7321, 
    "wssb_Colon_test_ssim_gt_h" : 0.8969, "wssb_Colon_test_ssim_gt_e" : 0.8560}, ignore_index=True)
df = df.append({"Name" : "BKSVD", 
    "wssb_Lung_test_ssim_gt_h" : 0.9764, "wssb_Lung_test_ssim_gt_e" : 0.9461, 
    "wssb_Breast_test_ssim_gt_h" : 0.9801, "wssb_Breast_test_ssim_gt_e" : 0.9632, 
    "wssb_Colon_test_ssim_gt_h" : 0.9826, "wssb_Colon_test_ssim_gt_e" : 0.9646}, ignore_index=True)
df["wssb_mean_test_ssim_gt_h"] = df[[c for c in df.columns if "ssim_gt_h" in c]].mean(axis=1)
df["wssb_mean_test_ssim_gt_e"] = df[[c for c in df.columns if "ssim_gt_e" in c]].mean(axis=1)
df["wssb_mean_test_ssim_gt"] = df[["wssb_mean_test_ssim_gt_h", "wssb_mean_test_ssim_gt_e"]].mean(axis=1)
df = highlight_columns(df, cols_idx=[1,2,3,4,5,6,7,8,9], mode="max")
df = df.transpose()
df.columns = df.iloc[0]
df = df.drop(df.index[0])
df.reset_index(inplace=True)
df.columns.name = None
df = df.drop(columns=["index"])
df["Organ"] = ["Lung", "Lung", "Breast", "Breast", "Colon", "Colon", "Mean", "Mean", "Mean"]
df["Stain"] = ["H", "E", "H", "E", "H", "E", "H", "E", "Mean"]

df.set_index(["Organ", "Stain"], inplace=True)
df

In [ ]:
styler = df.style
s = styler.to_latex(
    column_format=f"{ ''.join(['c' for i in range(len(df.columns)+2)]) }", clines="skip-last;data", 
    hrules = True, position_float="centering", multirow_align="c", sparse_index="True", 
    caption="Ground truth SSIM on WSSB dataset.", label="tab:gt_ssim_wssb",
    )
print(s)

In [ ]:
path = RESULTS_PATH + "results_wssb.csv"
df = pd.read_csv(path)
df = df.drop(columns=[c for c in df.columns if "loss" in c])
#df[[c for c in df.columns if "mse_rec" in c]]
df = df[df["pretraining_epochs"] == PRETRAINING_EPOCHS] 
df = df[df[hyperparam_name] == HYPERPARAM_VAL]
df = df[[c for c in df.columns if "mse_rec" in c]]
df["wssb_mean_test_mse_rec"] = df.mean(axis=1)
df["Name"] = [f"BCDnet p.e.={PRETRAINING_EPOCHS}, $\\theta={HYPERPARAM_VAL}$"]
df = df[["Name"] + list(df.columns[:-1])]
df = df.append({"Name" : "BCDnet (old)", "wssb_Lung_test_mse_rec" : 0.0066, "wssb_Breast_test_mse_rec" : 0.0172, "wssb_Colon_test_mse_rec" : 0.0092, "wssb_mean_test_mse_rec" : 0.0110}, ignore_index=True)
df = df.append({"Name" : "BKSVD", "wssb_Lung_test_mse_rec" : 0.0004, "wssb_Breast_test_mse_rec" : 0.0003, "wssb_Colon_test_mse_rec" : 0.0003, "wssb_mean_test_mse_rec" : 0.0003}, ignore_index=True)
df = highlight_columns(df, cols_idx=[1,2,3,4], mode="min")
df = df.transpose()
df.columns = df.iloc[0]
df = df.drop(df.index[0])
df.reset_index(inplace=True)
df.columns.name = None
df = df.drop(columns=["index"])
df["Organ"] = ["Lung", "Breast", "Colon", "Mean"]
df = df[["Organ"] + list(df.columns[:-1])]
df

In [ ]:
styler = df.style
styler = styler.hide(level=0, axis=0)  
s = styler.to_latex(
    column_format=f"{ ''.join(['c' for i in range(len(df.columns))]) }", clines="skip-last;data", 
    hrules = True, position_float="centering", multirow_align="c", sparse_index="True", 
    caption="Reconstruction MSE on WSSB dataset.", label="tab:mse_rec_wssb",
    )
print(s)